In [2]:
import pandas as pd
import numpy as np

players=pd.read_csv('players.csv')
transfers=pd.read_csv('transfers.csv')



In [ ]:
# Observamos datos
players.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,...,PKcon,Recov,GA,Saves,Save%,CS,CS%,PKA,PKsv,Season
0,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23,2000,20,13,1237,...,1,75,0,0,0,0,0,0,0,2023
1,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22,2000,30,14,1267,...,0,88,0,0,0,0,0,0,0,2023
2,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19,2003,7,1,101,...,0,5,0,0,0,0,0,0,0,2023
3,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17,2006,1,0,4,...,0,0,0,0,0,0,0,0,0,2023
4,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35,1987,31,31,2781,...,0,149,0,0,0,0,0,0,0,2023


In [5]:
transfers.head()

,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name
0,16136,2026-07-01,26/27,417,123,OGC Nice,Retired,NaN,500000.0,DANTE
1,1138758,2026-07-01,26/27,336,631,Sporting CP,Chelsea,52140000.0,45000000.0,GEOVANY QUENDA
2,195778,2026-06-30,25/26,79,27,VfB Stuttgart,Bayern Munich,0.0,12000000.0,ALEXANDER NÜBEL
3,569033,2026-06-30,25/26,39,27,1.FSV Mainz 05,Bayern Munich,0.0,4000000.0,ARMINDO SIEB
4,626913,2026-06-30,25/26,398,380,Lazio,Salernitana,0.0,15000000.0,BOULAYE DIA


In [ ]:
# Estandarizamos los nombres de los jugadores para el join
from unidecode import unidecode
players['Player']=players['Player'].str.upper()
transfers['player_name']=transfers['player_name'].str.upper()

transfers['player_name'] = transfers['player_name'].apply(lambda x: unidecode(str(x)))
players['Player'] = players['Player'].apply(lambda x: unidecode(str(x)))
players['Squad'] = players['Squad'].apply(lambda x: unidecode(str(x)))
transfers['from_club_name'] = transfers['from_club_name'].apply(lambda x: unidecode(str(x)))

In [ ]:
# Limpiamos transferencias sin costo y transformamos valores a millones
transfers=transfers[transfers['transfer_fee']>0]
transfers['transfer_fee']/=1000000
transfers['market_value_in_eur']/=1000000

In [ ]:
# Creamos la columna previous season para vincular la transferencia del jugador con uss estadisticas de la temporada pasada
transfers['season_year'] = transfers['transfer_season'].astype(str).str[:2].astype(int) + 2000
transfers['previous_season']=transfers['season_year']-1

In [ ]:
# Eliminamos elementos duplicados
players=players.drop_duplicates(subset=['Player','Season'],keep='last')

In [ ]:
# Realizamos el merge  en base a nombre y temporada
df=pd.merge(transfers,players,left_on=['player_name','previous_season'],right_on=['Player','Season'],how='inner')

In [11]:
df.head()

,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name,...,PKcon,Recov,GA,Saves,Save%,CS,CS%,PKA,PKsv,Season
0,234811,2025-07-01,25/26,150,336,Real Betis,Sporting CP,4.7,6.0,RUI SILVA,...,1,25,18,37,72.2,4,26.7,3,0,2024
1,263236,2025-07-01,25/26,826,148,Lens,Tottenham,25.0,25.0,KEVIN DANSO,...,1,59,0,0,0,0,0,0,0,2024
2,275218,2025-07-01,25/26,209,367,River Plate,Rayo Vallecano,1.6,6.0,AUGUSTO BATALLA,...,0,49,39,93,75.2,8,25,6,2,2024
3,283994,2025-07-01,25/26,39,3911,1.FSV Mainz 05,Stade Brestois,2.0,4.0,LUDOVIC AJORQUE,...,0,62,0,0,0,0,0,0,0,2024
4,287579,2025-07-01,25/26,430,36,Fiorentina,Fenerbahce,12.0,20.0,SOFYAN AMRABAT,...,0,7,0,0,0,0,0,0,0,2024


In [13]:
df.columns

Index(['player_id', 'transfer_date', 'transfer_season', 'from_club_id',
       'to_club_id', 'from_club_name', 'to_club_name', 'transfer_fee',
       'market_value_in_eur', 'player_name', 'season_year', 'previous_season',
       'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', 'MP',
       'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'xG', 'npxG',
       'xAG', 'Tkl', 'TklW', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Cmp%', 'PPA',
       'KP', 'PrgP', 'PrgC', 'Carries', 'Touches', 'Mis', 'Dis', 'PrgR',
       'CrdY', 'CrdR', 'PKwon', 'PKcon', 'Recov', 'GA', 'Saves', 'Save%', 'CS',
       'CS%', 'PKA', 'PKsv', 'Season'],
      dtype='object')

In [ ]:
# eliminamos columnas que no nos interesan o estan duplicadas
df.drop(['player_id','from_club_id','to_club_id','Player','Season'],axis=1,inplace=True)

In [15]:
df.head()

,transfer_date,transfer_season,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name,season_year,previous_season,Nation,...,PKwon,PKcon,Recov,GA,Saves,Save%,CS,CS%,PKA,PKsv
0,2025-07-01,25/26,Real Betis,Sporting CP,4.7,6.0,RUI SILVA,2025,2024,pt POR,...,0,1,25,18,37,72.2,4,26.7,3,0
1,2025-07-01,25/26,Lens,Tottenham,25.0,25.0,KEVIN DANSO,2025,2024,at AUT,...,0,1,59,0,0,0,0,0,0,0
2,2025-07-01,25/26,River Plate,Rayo Vallecano,1.6,6.0,AUGUSTO BATALLA,2025,2024,ar ARG,...,0,0,49,39,93,75.2,8,25,6,2
3,2025-07-01,25/26,1.FSV Mainz 05,Stade Brestois,2.0,4.0,LUDOVIC AJORQUE,2025,2024,fr FRA,...,2,0,62,0,0,0,0,0,0,0
4,2025-07-01,25/26,Fiorentina,Fenerbahce,12.0,20.0,SOFYAN AMRABAT,2025,2024,ma MAR,...,0,0,7,0,0,0,0,0,0,0


In [ ]:
# Arreglamos la columna de nacionalidad
df['Nation']=df['Nation'].str.split(' ').str[1]


In [17]:
df.shape

(1209, 54)

In [21]:
df.to_csv('../Analisis/base.csv',index=False)